In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
dir = '/programming/CS584/Movie_Recommender_System\data\processed
fn = dir + 'rating_csr.npz'
npy_file = np.load(fn)
csr = sparse.csr_matrix((npy_file['data'], npy_file['indices'], npy_file['indptr']), shape=npy_file['shape'])

FileNotFoundError: [Errno 2] No such file or directory: '/programming/CS584/Movie_Recommender_System/data/processedrating_csr.npz'

In [5]:
import os
from dask.distributed import Client
print(os.getcwd())

ImportError: No module named 'distributed'

In [4]:
def delete_csr_rows(csc, indices):
    bool_indx = np.ones(csc.shape[0], dtype=bool)
    len_string = "{} {} {}".format(csr.shape[0], indices.shape[0], bool_indx.shape[0])
    print(len_string)
    return  csr_matrix(csc[indices])

In [3]:
truncated_rows_csr = csr[csr.getnnz(1)>0]
print(truncated_rows_csr.shape)
print(csr.shape)

(2113, 65133)
(71534, 65133)


In [4]:
truncated_csr = truncated_rows_csr[:, truncated_rows_csr.getnnz(0)>0]
truncated_csr.shape

(2113, 9936)

In [5]:
df = pd.DataFrame(truncated_csr.todense())
df_nan = df.replace(0, np.NaN)
print(df_nan.tail())
print(df_nan.min().tail())

      0     1     2     3     4     5     6     7     8     9     ...   9926  \
2108   NaN   3.5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
2109   4.0   NaN   NaN   NaN   1.5   NaN   NaN   NaN   NaN   NaN  ...    NaN   
2110   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
2111   NaN   NaN   2.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
2112   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   4.5  ...    NaN   

      9927  9928  9929  9930  9931  9932  9933  9934  9935  
2108   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2109   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2110   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2111   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2112   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 9936 columns]
9931    2.5
9932    3.5
9933    3.0
9934    2.5
9935    3.0
dtype: float32


In [9]:
row_means = df_nan.mean(axis=1)
print(row_means.tail())

2108    3.564171
2109    3.361592
2110    4.008039
2111    3.420000
2112    4.148438
dtype: float32


In [10]:
col_means = df_nan.mean(axis=0)
print(col_means.tail())

9931    3.00
9932    3.50
9933    3.25
9934    2.50
9935    3.50
dtype: float32


In [11]:
df = df_nan

In [12]:
df_row_centered = df.sub(row_means, axis=0)
df_row_centered.tail()
print(df_row_centered.mean(axis=1).tail())

2108    2.078194e-07
2109    1.682955e-07
2110    7.206221e-07
2111   -7.629394e-08
2112    0.000000e+00
dtype: float32


In [13]:
df_col_centered = df.sub(col_means, axis=1)
df_col_centered.tail()
print(df_col_centered.mean(axis=0).tail())

9931    0.0
9932    0.0
9933    0.0
9934    0.0
9935    0.0
dtype: float32


In [1]:
test_movie_list = [[7,6,7,4,5,4],[6,7,0,4,3,4],[0,3,3,1,1,0],[1,2,2,3,3,4],[1,0,1,2,3,3]]
df_test = pd.DataFrame(test_movie_list)
df_test = df_test.replace(0, np.NaN)
df_test

NameError: name 'pd' is not defined

In [7]:
nulls = df_test.notnull()
trues_to_one = nulls.replace(True, 1)
trues_to_one

,0,1,2,3,4,5
0,True,True,True,True,True,True
1,True,True,False,True,True,True
2,False,True,True,True,True,False
3,True,True,True,True,True,True
4,True,False,True,True,True,True


In [96]:
print(nulls.loc[1])
print(nulls.loc[4])
print(sum(nulls.loc[2]&nulls.loc[1]))

0     True
1     True
2    False
3     True
4     True
5     True
Name: 1, dtype: bool
0     True
1    False
2     True
3     True
4     True
5     True
Name: 4, dtype: bool
3


In [8]:
common_element_list = np.zeros([len(nulls),len(nulls)])
for i in range(len(nulls)):
    for j in range(len(nulls)):
        common_element_list[i,j] = sum(nulls.loc[i] & nulls.loc[j])
print(common_element_list)

[[ 6.  5.  4.  6.  5.]
 [ 5.  5.  3.  5.  4.]
 [ 4.  3.  4.  4.  3.]
 [ 6.  5.  4.  6.  5.]
 [ 5.  4.  3.  5.  5.]]


In [102]:
nulls.loc[0].dot(nulls.loc[1])

True

In [13]:
import csv
output = 'data/processed/testoutput.csv'
with open(output, 'w') as results:
    wr = csv.writer(results)
    wr.writerow(common_element_list)